Grettings to :
https://github.com/PacktPublishing/Learn-OpenAI-Whisper/blob/main/Chapter07/LOAIW_ch07_2_Quantizing_Distil_Whisper_with_OpenVINO.ipynb

## Prerequisites

Before diving into the tutorial, ensure that you have the necessary prerequisites in place. This includes authenticating with the Hugging Face Hub using your token and verifying the authentication by running the provided code cells. These steps are crucial for accessing the required models and datasets throughout the tutorial.



In [1]:
!pip install --upgrade huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.7/450.7 kB 23.0 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


## Setting up a Hugging Face account

To access and utilize the vast array of datasets and machine learning models available on Hugging Face, an account is required. A Hugging Face account offers not just access to datasets like "PolyAI/minds14" but also allows for collaboration on projects, contribution to the community through dataset/model sharing, and even tracking progress on various machine learning tasks. Follow the steps outlined above to create your account and get started with Hugging Face.

## To create a Hugging Face account, you can follow these simple steps:
<img src="https://github.com/PacktPublishing/Learn-OpenAI-Whisper/raw/main/Chapter03/hugging_face_join.png" width=250>

1. Go to the Hugging Face website.
2. Click on “Sign Up” or go to https://huggingface.co/join
3. Enter your email address and password (you can skip the prompt to "Join organization").
4. If you already have an account, click on “Log in”.
5. To interact with the Hub, you need to be logged in with a Hugging Face account.

## To create a Hugging Face Access Token
<img src="https://github.com/PacktPublishing/Learn-OpenAI-Whisper/raw/main/Chapter03/hugging_face_settings_menu_option.png" width=650>
<img src="https://github.com/PacktPublishing/Learn-OpenAI-Whisper/raw/main/Chapter03/hugging_face_create_token.png" width=250>

1. Go to the Access Tokens tab in your Hugging Face settings.
2. Click on the New token button to create a new User Access Token.
3. Select the `write` role and a name for your token. This notebook only needs a `read` role; however, we will be writing to Hugging Face in [Chapter 4](https://colab.research.google.com/drive/1LADNomT0JUBCsopU6r_NsZfOaNiz2N3h).


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
# Verify authentication
from huggingface_hub import whoami
whoami()
# you should see something like {'type': 'user',  'id': '...',  'name': 'Wauplin', ...}

{'type': 'user',
 'id': '676187706ce06148154eb2f8',
 'name': 'aaiche',
 'fullname': 'aaiche',
 'isPro': False,
 'avatarUrl': '/avatars/1bf3857a4d2a4e5e6fb5381ba1ed7752.svg',
 'orgs': [],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'AA_TOKEN',
   'role': 'fineGrained',
   'createdAt': '2025-01-11T15:38:55.885Z',
   'fineGrained': {'canReadGatedRepos': False,
    'global': [],
    'scoped': [{'entity': {'_id': '676187706ce06148154eb2f8',
       'type': 'user',
       'name': 'aaiche'},
      'permissions': ['repo.content.read']}]}}}}

In [3]:
!pip install --upgrade pip
!pip install --upgrade transformers datasets[audio]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 55.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.2.2
    Uninstalling pip-22.2.2:
      Successfully uninstalled pip-22.2.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 128.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.9/780.9 kB 343.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 167.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 159.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 167.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 128.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.2
    Uninstalling tqdm-4.66.2:
      Successfully uninstalled tqdm-4.66.2
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled re

## Loading the PyTorch model

Loading the PyTorch Whisper model is a straightforward process using the transformers library. The `AutoModelForSpeechSeq2Seq.from_pretrained` method is employed to initialize the model. In this tutorial, we will use the `distil-whisper/distil-large-v2` model as the default example. Please note that the model will be downloaded during the first run, which may take some time.

However, you have the flexibility to choose from a variety of models available in the [Distil-Whisper Hugging Face collection](https://huggingface.co/collections/distil-whisper/distil-whisper-models-65411987e6727569748d2eb6). Some alternative options include `distil-whisper/distil-medium.en` and `distil-whisper/distil-small.en`. Additionally, models of the original Whisper architecture are also accessible, which you can explore further [here](https://huggingface.co/openai).

It's important to highlight the significance of preprocessing and post-processing in the model's usage. The `AutoProcessor` class, specifically the `WhisperProcessor`, plays a crucial role in preparing the audio input data for the model. It handles tasks such as converting the audio to a Mel-spectrogram representation and decoding the predicted output token IDs back into a string using the tokenizer.

To ensure a smooth and efficient workflow, the `AutoProcessor` class streamlines the preprocessing and post-processing steps, allowing you to focus on the core functionality of the Whisper model. By leveraging this class, you can easily integrate the Whisper model into your speech recognition pipeline, regardless of the specific model variant you choose.

In [4]:
import ipywidgets as widgets

model_ids = {
    "Distil-Whisper": [
        "distil-whisper/distil-large-v2",
        "distil-whisper/distil-medium.en",
        "distil-whisper/distil-small.en"
    ],
    "Whisper": [
        "openai/whisper-large-v3",
        "openai/whisper-large-v2",
        "openai/whisper-large",
        "openai/whisper-medium",
        "openai/whisper-small",
        "openai/whisper-base",
        "openai/whisper-tiny",
        "openai/whisper-medium.en",
        "openai/whisper-small.en",
        "openai/whisper-base.en",
        "openai/whisper-tiny.en",
    ]
}

model_type = widgets.Dropdown(
    options=model_ids.keys(),
    value="Whisper",
    description="Model type:",
    disabled=False,
)

model_type

Dropdown(description='Model type:', index=1, options=('Distil-Whisper', 'Whisper'), value='Whisper')

In [5]:
model_id = widgets.Dropdown(
    options=model_ids[model_type.value],
    value=model_ids[model_type.value][1],
    description="Model:",
    disabled=False,
)

model_id

Dropdown(description='Model:', index=1, options=('openai/whisper-large-v3', 'openai/whisper-large-v2', 'openai…

## Step 1: Loading the Transformers ASR Model

To begin building our speech recognition demo, we first need to have an Automatic Speech Recognition (ASR) model. You can either train your own model or use a pre-trained one. In this tutorial, we will leverage a pre-trained ASR model called "whisper" from OpenAI.

Loading the "whisper" model from the Hugging Face Transformers library is a straightforward process. Here's the code snippet to accomplish this:

```python
from transformers import pipeline
p = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")
```

With just these two lines of code, we initialize a pipeline for automatic speech recognition using the "openai/whisper-base.en" model. The pipeline abstracts away the complexities of working with the model directly, providing a high-level interface for performing ASR tasks.

By utilizing a pre-trained model like "whisper", we can quickly get started with building our demo without the need for extensive model training. This allows us to focus on integrating the model into our application and creating an engaging user experience.

## Step 2: Building a Full-Context ASR Demo with Transformers

Our first step in creating the speech recognition demo is to build a *full-context* ASR demo. In this demo, the user will speak the entire audio before the ASR model processes it and generates the transcription. Thanks to Gradio's intuitive interface, building this demo is a breeze.

We'll start by creating a function that wraps around the `pipeline` object we initialized earlier. This function will serve as the core of our demo, handling the audio input and generating the transcription.

To capture the user's audio input, we'll utilize Gradio's built-in `Audio` component. This component will be configured to accept input from the user's microphone and return the filepath of the recorded audio. For displaying the transcribed text, we'll use a simple `Textbox` component.

The `transcribe` function, which is the heart of our demo, takes a single parameter called `audio`. This parameter represents the audio data recorded by the user, stored as a NumPy array. However, the `pipeline` object expects the audio data to be in the `float32` format. To ensure compatibility, we first convert the audio data to `float32` and then normalize it by dividing it by its maximum absolute value. Finally, we pass the processed audio data to the `pipeline` object to obtain the transcribed text.

In [6]:
!ls -lah ~/.cache/huggingface/hub

ls: cannot access '/opt/app-root/src/.cache/huggingface/hub': No such file or directory


In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

processor = AutoProcessor.from_pretrained(model_id.value)

pt_model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id.value)
pt_model.eval();

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/4.29k [00:00<?, ?B/s]

In [9]:
!ls -lah ~/.cache/huggingface/hub

total 4.0K
drwxr-xr-x. 4 1000740000 root 79 Jan 13 16:30 .
drwxr-xr-x. 3 1000740000 root 51 Jan 13 16:30 ..
drwxr-xr-x. 3 1000740000 root 46 Jan 13 16:30 .locks
drwxr-xr-x. 6 1000740000 root 65 Jan 13  2025 models--openai--whisper-large-v2
-rw-r--r--. 1 1000740000 root  1 Jan 13 16:30 version.txt


In [10]:
import torch
this_device = "cuda" if torch.cuda.is_available() else "cpu"
this_device
print(f"Using device: {this_device}")

Using device: cpu


In [11]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

### Preparing the Input Sample

To use the Whisper model for speech recognition, we need to properly prepare the input audio sample. The `WhisperProcessor` expects the audio data to be in the form of a NumPy array, along with information about the audio sampling rate. It then processes the audio and returns the `input_features` tensor, which is used for making predictions.

The conversion of the audio file to the required NumPy format is conveniently handled by the Hugging Face Datasets library. This library provides a seamless interface for loading and preprocessing audio data, making it easier to integrate with the Whisper model.

To prepare the input sample, the next Python code:

1. Loads the audio file using the Hugging Face Datasets library.
2. Extracts the audio data as a NumPy array and obtain the sampling rate.
3. Passes the audio array and sampling rate to the `WhisperProcessor`.
4. Retrieves the `input_features` tensor from the processor.

In [12]:
!ls -lah ~/.cache/huggingface/hub

total 4.0K
drwxr-xr-x. 4 1000740000 root 79 Jan 13 16:30 .
drwxr-xr-x. 3 1000740000 root 51 Jan 13 16:30 ..
drwxr-xr-x. 3 1000740000 root 46 Jan 13 16:30 .locks
drwxr-xr-x. 6 1000740000 root 65 Jan 13 16:31 models--openai--whisper-large-v2
-rw-r--r--. 1 1000740000 root  1 Jan 13 16:30 version.txt


In [13]:
!ls -lah ~/.cache/huggingface/datasets

ls: cannot access '/opt/app-root/src/.cache/huggingface/datasets': No such file or directory


### dataset:
    "distil-whisper/librispeech_long"

In [14]:
from datasets import load_dataset

def extract_input_features(sample):
    input_features = processor(
        sample["audio"]["array"],
        sampling_rate=sample["audio"]["sampling_rate"],
        return_tensors="pt",
    ).input_features
    return input_features

dataset = load_dataset(
    "distil-whisper/librispeech_long", "clean", split="validation"
)
sample = dataset[0]
input_features = extract_input_features(sample)

README.md:   0%|          | 0.00/480 [00:00<?, ?B/s]

(…)-00000-of-00001-913508124a40cb97.parquet:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

In [15]:
sample

{'audio': {'path': '0d38672e0bbdbdc460af55b8bb84a15b2730db2819f2af64f9c777d4d586f2de',
  'array': array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00024414, 0.00048828,
         0.0005188 ]),
  'sampling_rate': 16000}}

In [16]:
input_features

tensor([[[ 1.1933e-01, -9.4576e-02, -1.0978e-01,  ...,  2.2357e-01,
           2.7244e-01,  4.1318e-01],
         [ 4.9347e-04, -8.9271e-02, -6.7290e-02,  ...,  4.2127e-01,
           4.6272e-01,  4.1557e-01],
         [-1.5326e-01, -2.0804e-01, -2.2227e-01,  ...,  8.5404e-01,
           8.3540e-01,  8.4285e-01],
         ...,
         [-6.3673e-01, -6.3673e-01, -6.3673e-01,  ..., -1.8856e-01,
          -1.9991e-01, -4.2395e-02],
         [-6.3673e-01, -6.3673e-01, -6.3673e-01,  ..., -6.8221e-02,
          -1.7072e-01, -4.4241e-02],
         [-6.3673e-01, -6.3673e-01, -6.3673e-01,  ..., -1.1030e-01,
          -1.8742e-01, -5.2066e-02]]])

In [17]:
!ls -lah ~/.cache/huggingface/datasets

total 0
drwxr-xr-x. 3 1000740000 root 192 Jan 13 16:32 .
drwxr-xr-x. 4 1000740000 root  67 Jan 13 16:32 ..
drwxr-xr-x. 3 1000740000 root  19 Jan 13 16:32 distil-whisper___librispeech_long
-rw-r--r--. 1 1000740000 root   0 Jan 13 16:32 _opt_app-root_src_.cache_huggingface_datasets_distil-whisper___librispeech_long_clean_0.0.0_164d3b41852b1eebe89f1dc0e6e0042f16835ea0.lock


### Running Model Inference

With the input sample prepared, we can now perform speech recognition using the Whisper model. The model provides a convenient `generate` interface that simplifies the inference process. Here's how you can run the model inference:

1. Pass the `input_features` tensor to the `generate` method of the Whisper model.
2. The model will process the input and generate the predicted token IDs.
3. Once the generation is complete, use the `processor.batch_decode` method to decode the predicted token IDs into human-readable text transcription.

The `generate` method handles the complex task of sequence generation, taking into account the model's architecture and the provided input features. It produces the predicted token IDs, which represent the transcribed text in a encoded format.

By leveraging the `generate` interface and the `processor.batch_decode` method, you can easily perform speech recognition with the Whisper model. The model takes care of the complex task of mapping the audio input to text output, while the processor handles the necessary decoding step to provide you with the final transcription.

In [18]:
import warnings
warnings.filterwarnings("ignore")

In [19]:
%%time
import IPython.display as ipd

predicted_ids = pt_model.generate(input_features)
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

#display(ipd.Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"]))
# aa: sample doesnt have 
#print(f"Reference: {sample['text']}")
#print(f"Result: {transcription[0]}")

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


CPU times: user 4min 4s, sys: 22.3 s, total: 4min 26s
Wall time: 45.3 s


In [20]:
print(f"Result: {transcription[0]}")

Result:  Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his manner. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Leighton's work is really Greek after all, and can discover


In [21]:
transcription

[" Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his manner. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Leighton's work is really Greek after all, and can discover"]

In [22]:
!pip install --upgrade pip
!pip install --upgrade transformers datasets[audio] accelerate

###  Usage 

In [23]:
import warnings
warnings.filterwarnings("ignore")

In [24]:
%%time
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=False, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=torch_dtype,
        device=device,
    
        max_new_tokens=128, # aa : added to make it work
        chunk_length_s=30,  # aa
        batch_size=16,        # aa
        return_timestamps=True  # aa
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

result = pipe(sample)
print(result["text"])


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Device set to use cpu


 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Leighton's work is really Greek after all, and can discover in it but little of rocky Ithaca. Linnell's pictures are a sort of Upguards and Adam paintings, and Mason's exquisite idylls are as national as a jingo poem. Mr. Burkett Foster's landscapes smile at one much in the same way that Mr. Carker used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap on the back before he says, like a shampooer in a Turkish bath, Next man!
CPU times: user 9min 33s, sys: 1min 43s, total: 11min 17s
Wall time: 3min 24s


 ### Chunked Long-Form 

In [27]:
%%time
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "openai/whisper-large-v3"

# aa : low_cpu_mem_usage from True to False
model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=False
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        chunk_length_s=30,
        batch_size=16,  # batch size for inference - set based on your device
        torch_dtype=torch_dtype,
        device=device,
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

result = pipe(sample)
print(result["text"])


Device set to use cpu


 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Leighton's work is really Greek after all, and can discover in it but little of rocky Ithaca. Linnell's pictures are a sort of Upguards and Adam paintings, and Mason's exquisite idylls are as national as a jingo poem. Mr. Burkett Foster's landscapes smile at one much in the same way that Mr. Carker used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap on the back before he says, like a shampooer in a Turkish bath, Next man!
CPU times: user 7min 36s, sys: 30.4 s, total: 8min 6s
Wall time: 1min 26s


### Torch compile 

###  Flash Attention 2 
    to be tested on GPU

###  Torch Scale-Product-Attention (SDPA) 
    to be tested on GPU